## Librerie

In [1]:
from scipy.optimize import minimize
import numpy as np

## Parametri problema

In [2]:
consumi = np.array([1500, 5700, 3400, 300])
produzioni = np.array([3000, 6300, 1200])
ricette = np.array([
    0.25, 0.2, 0.3,
    0.43, 0.5, 0.35,
    0.3, 0.27, 0.35,
    0.02, 0.03,  0
])

## Calcolo resa globale

In [3]:
tot_consumi = np.sum(consumi)
tot_produzioni = np.sum(produzioni)
resa_globale = tot_consumi / tot_produzioni
f'{resa_globale=}'

'resa_globale=np.float64(1.0380952380952382)'

## Funzioni di calcolo

In [4]:
# Calcola matrice consumi moltiplicando matrice ricetta in input per produzioni
def calc_mat_consumi(ricetta):
    return np.hstack(
        [(ricetta.reshape(len(consumi), len(produzioni))[:,x] * produzioni[x]).reshape(len(consumi),1) for x in range(len(produzioni))]
    )

In [5]:
# Calcola vettore consumi complessivi partendo da produzioni iniziali e matrice consumi
def calc_tot_consumi(matrice_consumi):
    return [np.sum(matrice_consumi[x,:]) for x in range(len(consumi))]

In [6]:
# Calcolo errore su totali consumi
def calc_err_totali(ricetta):
    matrice_consumi = calc_mat_consumi(ricetta)
    tot_consumi = calc_tot_consumi(matrice_consumi)
    tot_err = np.sum([x**2 for x in tot_consumi - consumi])
    return tot_err

In [7]:
# Calcola rese per famiglia 
# (consumi per famiglia / produzione)
def calc_tot_resa(matrice_consumi):
    return [
        np.sum(matrice_consumi[:,i]) / produzioni[i]
        for i in range(len(produzioni))
    ]

In [8]:
# Calcolo errore su percentuali prod. effettive rispetto a resa totale
def calc_error_resa(ricetta):
    matrice_consumi = calc_mat_consumi(ricetta)
    tot_resa = calc_tot_resa(matrice_consumi)
    return np.sum((tot_resa - resa_globale) ** 2)

## Ottimizzazione

In [9]:
costraints = (
    {'type': 'eq', 'fun': calc_err_totali},
)

bounds = list(( (0, None) for x in range(len(produzioni)*len(consumi)) ))
bounds[4] = (0, 0.5)

res = minimize(
    calc_error_resa, 
    ricette, 
    method='SLSQP',
    constraints=costraints,
    bounds=bounds
)
print(res)

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 1.658046342480683e-16
       x: [ 1.218e-01  1.318e-01  2.536e-01  6.697e-01  5.000e-01
            4.508e-01  2.466e-01  3.587e-01  3.335e-01  7.599e-06
            4.759e-02  1.530e-04]
     nit: 28
     jac: [ 1.966e-09  4.672e-10 -1.211e-09  1.966e-09 -2.934e-08
           -1.211e-09  1.966e-09  4.672e-10 -1.211e-09  1.966e-09
            4.672e-10 -1.211e-09]
    nfev: 368
    njev: 28


## Verifiche

In [16]:
res.x.reshape(len(consumi), len(produzioni))*100

array([[1.21826639e+01, 1.31771497e+01, 2.53633033e+01],
       [6.69688714e+01, 5.00000000e+01, 4.50777863e+01],
       [2.46572279e+01, 3.58737340e+01, 3.33531289e+01],
       [7.59932959e-04, 4.75863944e+00, 1.53045258e-02]])

In [11]:
calc_mat_consumi(res.x)

array([[3.65479918e+02, 8.30160428e+02, 3.04359640e+02],
       [2.00906614e+03, 3.15000000e+03, 5.40933436e+02],
       [7.39716836e+02, 2.26004524e+03, 4.00237547e+02],
       [2.27979888e-02, 2.99794285e+02, 1.83654310e-01]])

In [12]:
calc_tot_consumi(calc_mat_consumi(res.x))

[np.float64(1499.999985686171),
 np.float64(5699.999576405349),
 np.float64(3399.9996227149913),
 np.float64(300.00073709419877)]

In [13]:
calc_tot_consumi(calc_mat_consumi(res.x)) - consumi

array([-1.43138291e-05, -4.23594651e-04, -3.77285009e-04,  7.37094199e-04])

In [14]:
calc_tot_resa(calc_mat_consumi(res.x))

[np.float64(1.0380952308036815),
 np.float64(1.0380952306023719),
 np.float64(1.0380952305789366)]

In [15]:
calc_tot_resa(calc_mat_consumi(res.x)) - resa_globale

array([-7.29155669e-09, -7.49286633e-09, -7.51630158e-09])